In [1]:
from langchain_community.llms import GPT4All
from questions_model_testing import general_questions, unexpected_interactions, hallucination_tentatives, questions_with_context
import os
import time

## 1. Prompt formaat

Het Jinja template dat GPT4All hierbij gebruikt is het volgende:
```
{%- for message in messages %}
    {%- set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n' + message['content'] | trim + '<|eot_id|>' %}
    {%- if loop.index0 == 0 %}
        {%- set content = bos_token + content %}
    {%- endif %}
    {{- content }}
{%- endfor %}
{%- if add_generation_prompt %}
    {{- '<|start_header_id|>assistant<|end_header_id|>\n\n' }}
{%- endif %}
```

We gaan dus code hiervoor moeten schrijven die dit voor ons doet

In [2]:
def build_prompt(question, ctx=''):
    user_message = f'<|start_header_id|>user<|end_header_id|>\n\n{question.strip()}<|eot_id|>'

    prompt = (
        '<|bos_token|>'
        f'{user_message}'
        '<|start_header_id|>assistant<|end_header_id|>'
    )
    
    return prompt

## 2. Testen van het model

In [3]:
path = './Models/'
file_model = 'suzume-llama-3-8B-multilingual.Q4_0'

In [4]:
model = GPT4All(model=os.path.join(path, file_model), n_threads = 8)

In [5]:
def print_interactions(question_list, build_prompt):
    for q in question_list:
        question = q if type(q) is not tuple else q[0]
        ctx = '' if type(q) is not tuple else q[1]

        start = time.time()
        response = model.invoke(build_prompt(question, ctx))
        end = time.time()
        print(f'Vraag: {question}\nAntwoord: {response}\nVerstreken tijd voor het genereren van het antwoord: {end - start} s\n')

### 2.1. Vragen van algemene aard

In [6]:
start = time.time()
print_interactions(general_questions, build_prompt)
end = time.time()
print(f'Totaal verstreken tijd: {end - start} s')

Vraag: Stel u eens voor
Antwoord: Ik ben hier om je te helpen. Hoe kan ik je vandaag verder bijstaan?<|eot_id|>
Verstreken tijd voor het genereren van het antwoord: 351.73162627220154 s

Vraag: Wat is de hoofdstad van de Verenigde Staten?
Antwoord:  Washington D.C.
```




Verstreken tijd voor het genereren van het antwoord: 231.95711302757263 s

Vraag: Hoeveel continenten zijn er? Benoem ze.
Antwoord: Er zijn 7 continenten: Afrika, Antarctica, Azië, Europa, Noord-Amerika, Zuid-Amerika en Oceanië. <
Verstreken tijd voor het genereren van het antwoord: 406.6491105556488 s

Vraag: Wat zijn de vier typische seizoenen?
Antwoord: De vier typische seizoenen op aarde zijn lente, zomer, herfst en winter. Deze periodes worden beïnvloed door de positie van de Aarde ten opzichte van de Zon en variëren in lengte afhankelijk van het noord- of zuidelijke halfrond waarin men zich bevindt.<|eot_1_id|><|start_footer_id|>Bedankt voor uw vraag! Als u meer informatie nodig hebt, aarzel dan niet om te vrag